In [1]:
from web3 import Web3

Configuring Web3.py to communicate with an Ethereum node.

Option to use IPC, HTTP, or Websocket providers.

The legit full way is:
+ Download an Ethereum node (such as Geth)
+ Start Geth in one terminal and wait for it to sync the network. The default HTTP port is 8545, but is configurable.
+ Tell Web3.py to connect to the node via HTTP on localhost:8545 by `w3 = Web3(Web3.HTTPProvider('http:localhost:8545`
+ Use the w3 instance to interact with the node.

For tutorial purposes, we are going to use `EthereumTesterProvider`, which is a simulated ethereum node with fake currency.

In [3]:
# configuring w3.py to use tester node
w3 = Web3(Web3.EthereumTesterProvider())

In [4]:
w3

/opt/homebrew/Caskroom/miniconda/base/envs/default/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# sanity check
w3.isConnected()

True

In [8]:
# public addresses; like account number
w3.eth.accounts

['0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf',
 '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF',
 '0x6813Eb9362372EEF6200f3b1dbC3f819671cBA69',
 '0x1efF47bc3a10a45D4B230B5d10E37751FE6AA718',
 '0xe1AB8145F7E55DC933d51a18c793F901A3A0b276',
 '0xE57bFE9F44b819898F47BF37E5AF72a0783e1141',
 '0xd41c057fd1c78805AAC12B0A94a405c0461A6FBb',
 '0xF1F6619B38A98d6De0800F1DefC0a6399eB6d30C',
 '0xF7Edc8FA1eCc32967F827C9043FcAe6ba73afA5c',
 '0x4CCeBa2d7D2B4fdcE4304d3e09a1fea9fbEb1528']

In [11]:
# see balance
balance = w3.eth.get_balance(w3.eth.accounts[0]) # units in wei

In [12]:
balance

1000000000000000000000000

In [13]:
# convert wei to eth
w3.fromWei(balance, 'ether')

Decimal('1000000')

## Block Data

In [14]:
# what is inside a block?
w3.eth.get_block('latest')

AttributeDict({'number': 0,
 'hash': HexBytes('0x7ad665ee43016c92b04999b8c40eefa1cb37123e9cc7e7564c7314e1b530d92d'),
 'parentHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x000000000000002a'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logs_bloom': 0,
 'transactionsRoot': HexBytes('0x56e81f171bcc55a6ff8345e692c0f86e5b48e01b996cadc001622fb5e363b421'),
 'receipts_root': '0x56e81f171bcc55a6ff8345e692c0f86e5b48e01b996cadc001622fb5e363b421',
 'stateRoot': HexBytes('0xf1588db9a9f1ed91effabdec31f93cb4212b008c8b8ba047fd55fabebf6fd727'),
 'miner': '0x0000000000000000000000000000000000000000',
 'difficulty': 131072,
 'totalDifficulty': 131072,
 'size': 511,
 'extraData': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'gasLimit': 3141592,
 'gasUsed': 0,
 'timestamp': 1626942375,
 'transactions': [],
 'uncles': []})

In this simulation environment, blocks are not mined every 15 seconds. So we are looking at a brand new genesis block. You can tell that it is a genesis block because its `parentHash` is all zeros.

## First Transaction

Sendiing a few test ether from one account to another. This will trigger mining.

In [16]:
tx_hash = w3.eth.send_transaction({
    'from': w3.eth.accounts[0],
    'to': w3.eth.accounts[1],
    'value': w3.toWei(3, 'ether')
})

In the real world, you would have to wait several seconds until this gets mined (confirmed). In test environment, it is immediate.

In [17]:
# checking if transaction has been mined:
w3.eth.wait_for_transaction_receipt(tx_hash)

AttributeDict({'transactionHash': HexBytes('0x15e9fb95dc39da2d70f4cc41556bd092c68a97a04892426a064e321bfe78662a'),
 'transactionIndex': 0,
 'blockNumber': 1,
 'blockHash': HexBytes('0x3780f9ec642491d5d702c710bf03622db84a4df774975861a36de7afb97ff962'),
 'cumulativeGasUsed': 21000,
 'gasUsed': 21000,
 'contractAddress': None,
 'logs': [],
 'status': 1})

In [18]:
# viewing completed transaction
w3.eth.get_transaction(tx_hash)

AttributeDict({'hash': HexBytes('0x15e9fb95dc39da2d70f4cc41556bd092c68a97a04892426a064e321bfe78662a'),
 'nonce': 0,
 'blockHash': HexBytes('0x3780f9ec642491d5d702c710bf03622db84a4df774975861a36de7afb97ff962'),
 'blockNumber': 1,
 'transactionIndex': 0,
 'from': '0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf',
 'to': '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF',
 'value': 3000000000000000000,
 'gas': 121000,
 'gasPrice': 1,
 'data': '0x',
 'v': 28,
 'r': HexBytes('0x11bebd35f91582f55dc180dcfc1c5ccad48dadc207802727f7ac997df6490b22'),
 's': HexBytes('0x697db707f5b7cc4d3a3196b434b0d5616300b8afbe8a21ab47ed9335252e4ebd')})

In [19]:
# verify success of transaction by checking account balances
w3.eth.get_balance(w3.eth.accounts[0])

999996999999999999979000

Notice how 21000 additional wei have been spent as gas fees.

In [20]:
w3.eth.get_balance(w3.eth.accounts[1])

1000003000000000000000000